# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Create cell

In [ ]:
import gunay_model
from neuron import h
import bluepyopt.ephys as ephys

cell = gunay_model.define_cell() # cell model description
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)
cell.instantiate(sim=nrnsim)
icell = cell.icell # cell instantiated in Hoc

In [ ]:
named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
for k, v in named_seclists.items():
    if len(v)==0:
        named_seclists.pop(k)
    else:
        print("{} : {} sections".format(k, len(v)))

In [ ]:
soma = named_seclists['somatic'][0]
axon = named_seclists['axonal'][0]
dend = named_seclists['basal'][0]

# Plot cell morphology

In [ ]:
from common import morphology
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
# h.define_shape(sec=soma) # for cells without 3D morphology
morphology.shapeplot(h, ax)

# Plot soma locations
morphology.mark_locations(h, soma, 0.5, label='soma', markspec='or')
morphology.mark_locations(h, axon, 1.0, label='axon_tip', markspec='og')


plt.legend()
plt.show()

# Experiment: spontaneous activity

In [ ]:
# start recording
vrec = h.Vector()
vrec.record(dend(0.5)._ref_v)
trec = h.Vector()
trec.record(h._ref_t)

# Record currents
from common import analysis
rec_secs = {
    'soma': soma,
    'dend': dend,
}
trace_specs = {
#     'iCaH': {'var':'iCaH', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
#     'iHCN': {'var':'iHCN', 'mech':'HCN', 'sec':'dend', 'loc':0.5},
#     'iHCN2': {'var':'iHCN', 'mech':'HCN2', 'sec':'dend', 'loc':0.5},
#     'iKCNQ': {'var':'iKCNQ', 'mech':'KCNQ', 'sec':'dend', 'loc':0.5},
#     'iKv2': {'var':'iKv2', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
#     'iKv3': {'var':'iKv3', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
#     'iKv4f': {'var':'iKv4f', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
#     'iKv4s': {'var':'iKv4s', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
#     'iNa': {'var':'iNa', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
#     'iNa': {'var':'iNa', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
#     'iSK': {'var':'iSK', 'mech':'SK', 'sec':'dend', 'loc':0.5},
    # gating variables
    'iCaH': {'var':'m', 'mech':'CaHVA', 'sec':'dend', 'loc':0.5},
    'iHCN': {'var':'m', 'mech':'HCN', 'sec':'dend', 'loc':0.5},
    'iHCN2': {'var':'m', 'mech':'HCN2', 'sec':'dend', 'loc':0.5},
    'iKCNQ': {'var':'m', 'mech':'KCNQ', 'sec':'dend', 'loc':0.5},
    'mKv2': {'var':'m', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
    'hKv2': {'var':'h', 'mech':'Kv2', 'sec':'dend', 'loc':0.5},
    'mKv3': {'var':'m', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
    'hKv3': {'var':'h', 'mech':'Kv3', 'sec':'dend', 'loc':0.5},
    'mKv4f': {'var':'m', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
    'hKv4f': {'var':'h', 'mech':'Kv4f', 'sec':'dend', 'loc':0.5},
    'mKv4s': {'var':'m', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
    'hKv4s': {'var':'h', 'mech':'Kv4s', 'sec':'dend', 'loc':0.5},
    'mNa': {'var':'m', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
    'hNa': {'var':'h', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
    'sNa': {'var':'s', 'mech':'NaF', 'sec':'dend', 'loc':0.5},
    'mNaP': {'var':'m', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
    'hNaP': {'var':'h', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
    'sNaP': {'var':'s', 'mech':'NaP', 'sec':'dend', 'loc':0.5},
    'mSK': {'var':'m', 'mech':'SK', 'sec':'dend', 'loc':0.5},
}
rec_dt = 0.05
vec_dict, _ = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 500.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# plot results
plt.figure()
plt.plot(trec.as_numpy(), vrec.as_numpy())
plt.xlim(0, h.tstop)
# plt.ylim(-80,60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),

plt.show()

In [ ]:
print(min(vrec.as_numpy()))
print(max(vrec.as_numpy()))

# i_sk = vec_dict['iSK']
# i_sk[0:30]

# i_hcn = vec_dict['iHCN']
# i_hcn[0:30]

# i_kv3 = vec_dict['iKv3']
# i_kv3[0:30]

m_kv3 = vec_dict['mKv3']
m_kv3[0:30]
print(min(m_kv3))
print(max(m_kv3))

m_sk = vec_dict['mSK']
m_sk[0:30]
print(min(m_sk))
print(max(m_sk))